In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv(r'C:/Users/heave/Project/question1_sentiment.csv')
print(data.info())
print(data.head())
print(data.describe())
print(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id        25000 non-null int64
label     25000 non-null int64
review    25000 non-null object
dtypes: int64(2), object(1)
memory usage: 586.0+ KB
None
   id  label                                             review
0   1      1  With all this stuff going down at the moment w...
1   2      1  \The Classic War of the Worlds\" by Timothy Hi...
2   3      0  The film starts with a manager (Nicholas Bell)...
3   4      0  It must be assumed that those who praised this...
4   5      1  Superbly trashy and wondrously unpretentious 8...
                 id         label
count  25000.000000  25000.000000
mean   12500.500000      0.198880
std     7217.022701      0.747896
min        1.000000     -1.000000
25%     6250.750000      0.000000
50%    12500.500000      0.000000
75%    18750.250000      1.000000
max    25000.000000      1.000000
          id  label                                